<a href="https://colab.research.google.com/github/AyushPathak7074/ChatbotAssistant/blob/master/ChatbotAssist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
#libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

#libraries needed for tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
# load the intent.json files from your local device
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{\r\n        "intents": [\r\n                {\r\n                        "tag": "greeting",\r\n                        "patterns": [\r\n                                "Hi",\r\n                                "How are you",\r\n                                "Is anyone there?",\r\n                                "Hello",\r\n                                "Good day"\r\n                        ],\r\n                        "responses": [\r\n                                "Hello, thanks for visiting",\r\n                                "Good to see you again",\r\n                                "Hi there, how can I help?"\r\n                        ],\r\n                        "context_set": ""\r\n                },\r\n                {\r\n                        "tag": "goodbye",\r\n                        "patterns": [\r\n                                "Bye",\r\n                                "See you later",\r\n                                "Goodbye"\r\n     

In [54]:
#import our chat-bot intents file
with open('intents.json') as json_data:
  intents= json.load(json_data)

In [55]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [57]:
words=[]
classes=[]
documents=[]
ignore=['?']
#loop through each sentence in the intent's patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tockenize each and every word in the sentence
    w=nltk.word_tokenize(pattern)
    #add word to the word list
    words.extend(w)
    #add words to documents
    documents.append((w,intent['tag']))
    #add tags to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


In [58]:
#perform stemming and lower each word as well as remove duplicates
words=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words= sorted(list(set(words)))

#remove duplicate classes
print(len(documents),"documents")
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words", words)

27 documents
8 classes ['greeting', 'goodbye', 'thanks', 'chatbot', 'location', 'connect', 'movies', 'about']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [59]:
#create training data
training=[]
output=[]
#create an empty array for output
output_empty = [0]* len(classes)
#create training set , bag of words for each sentence
for doc in documents:
  #initialize bag of words
  bag=[]
  #list of tokenize words for the pattern
  pattern_words= doc[0]
  #stemming each word
  pattern_words= [stemmer.stem(word.lower()) for word in pattern_words]
  #create a bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    #output is 1 for current tag and 0 for other tag
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1

    training.append([bag,output_row])

#shuffling features and turning it into np.array
random.shuffle(training)
training= np.array(training)

#creating training list
train_x= list(training[:,0])
train_y= list(training[:,1])


<ipython-input-59-67772d4dde97>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training= np.array(training)


In [60]:
model= tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]),activation='softmax'))
model.compile(tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuray'])

In [61]:
model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size=8,verbose=1)
model.save("model.pkl")

Epoch 1/100


TypeError: ignored